In [1]:
from typing import List
from config.endpoints import MAINNET_BASE_URL
from helpers.utility import _make_get_request, _make_post_request
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
import requests
import pandas as pd
import os



class TransactionsAPI:
    def __init__(self, api_key: str):
        self.base_url = MAINNET_BASE_URL
        self.headers = {"Authorization": f"Bearer {api_key}"}
        #self.api_key_query = f"?api-key={api_key}"

    def _build_url(self, path: str, params: dict = None) -> str:
        url = self.base_url + path
        if params:
            query_string = "&".join(f"{k}={v}" for k, v in params.items() if v)
            url += f"?{query_string}"
        return url


    def get_raw_transactions(
        self, 
        accounts: List[str], 
        start_slot: int=None,
        end_slot: int=None,
        start_time: int=None,
        end_time: int=None,
        limit: int=None,
        sort_order: str=None,
        pagination_token: str=None
        ):

        path = "/v1/raw-transactions"
        url = self._build_url(path)
        payload = {
            "query": {
                "accounts": accounts,
                "startSlot": start_slot,
                "endSlot": end_slot,
                "startTime": start_time,
                "endTime": end_time
            },
            "options": {
                "limit": limit,
                "sortOrder": sort_order,
                "paginationToken": pagination_token
            }
        }
        return _make_post_request(url, payload, headers=self.headers)

    def get_parsed_transactions(self, transactions: List[str], commitment: str = None):
        path = "/v0/transactions"
        url = self._build_url(path, {"commitment": commitment})
        payload = {"transactions": transactions}
        return _make_post_request(url, payload, headers=self.headers)

    def get_parsed_transaction_history(
        self, 
        address: str, 
        before: str='', 
        until: str='', 
        commitment: str='',
        source: str='',
        type: str=''
        ):
        
        path = f"/v0/addresses/{address}/transactions"
        params = {
            "before": before,
            "until": until,
            "commitment": commitment,
            "source": source,
            "type": type
        }

        url = self.base_url + path
        return _make_get_request(url, params=params)

In [2]:
HELIUS_API_KEY = os.getenv('api_key')

In [3]:
# helpers/utility.py
def _make_post_request(url: str, payload: dict, headers: dict) -> dict:
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error: {response.status_code}: {response.text}")

In [4]:
# Initialize with your API key (ensure it's valid and active)
#HELIUS_API_KEY = os.getenv("api_key")  
tx_api = TransactionsAPI(api_key=HELIUS_API_KEY)

tx_api

In [6]:
# # Example usage
# transactions = [""]  # Your transaction signatures
# response = tx_api.get_parsed_transactions(transactions)
# print(response)

In [7]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api

In [8]:
balances_api.get_balances("hq8fC9VkxU6rv1QvMUkGKbhFR3MxSd7nCkCGaaXx9MA")

{'tokens': [{'tokenAccount': '5iPPv6kqUA5BLgkQRwChbWNa17o14dU1GS9ksxgZfXG1',
   'mint': 'GRUmPYbiTpq9ZPy5LAqBMMze7kErf5dEX2i9qYfwoSmR',
   'amount': 0,
   'decimals': 9},
  {'tokenAccount': 'AvnWQTMCokoRR9wd8pYJwA5JbTLS3ZX97fSNDqD1g8v7',
   'mint': '3DH8WZ6C2HdDR4J9ekYnTfai8Tx8mxn92mW8zcYmP4px',
   'amount': 354000000000,
   'decimals': 9},
  {'tokenAccount': '37vryqQ6z4j7sdAJjTMAmzVKDkadzi4ZGcVoPkoToVqb',
   'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
   'amount': 33378,
   'decimals': 6},
  {'tokenAccount': '8kuTaQgk46Wo7PCuXfZuySEP85fZRgVbvCXkAfwVmt9S',
   'mint': 'Hk82ay38uGpYgqjdXXCiATavVQwAMaBNyxsUWbapmKuL',
   'amount': 1000000000,
   'decimals': 6},
  {'tokenAccount': 'GXnj7rSQtBUWcQbeP5QEyKJyv4br64uckb8xsS1sBY11',
   'mint': 'G4hdrdhnboNgFXo1uP21VwZ9zrNbxwZDmtpZt2Mnpump',
   'amount': 12126567051,
   'decimals': 6},
  {'tokenAccount': '6XWCWkwCdbBTzK5GrtTCVRQqXaB2QGuDHqZsXEt9KbBo',
   'mint': 'EWx3xDR76YfxF3VHwTLgwU9WYrwANw3yxX5vvk8Kpump',
   'amount': 119000000,
 

In [ ]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api